In [ ]:
%load_ext autoreload
%autoreload 2
import sys, warnings, time, numpy, yaml, pandas
sys.path.append("../src/") # go to parent dir
from data_access import get_X, get_y, get_train_test
from models.factory import ModelFactory
from models.xgboost_model import XgboostModel
from models.sklearn_model import SklearnModel
warnings.filterwarnings('ignore')

In [ ]:
feature=['HOME_WINS', 'DRAW', 'AWAY_WINS']
train_data = get_X('train')
train_scores = get_y()
test_data = get_X('test')
X_train, y_train, X_test, y_test, target = get_train_test(train_size=0.8, random_state=42, feature=feature)

In [ ]:
print(f'y_train={y_train.head()}')
print(f'y_test={y_test.head()}')
print(f'target={target.head()}')

In [ ]:
def eval_model(model, save_model=False):
    start = time.time()
    model.train()
    score = model.evaluate(X_test)
    end = time.time()
    print(f'{model.name}={score} in {numpy.round((end-start), 2)}s')
    if save_model:
        model.save(test_data)
    return {'name': model.name, 'score': score, 'time': numpy.round((end-start), 2)}

In [ ]:
%load_ext autoreload
%autoreload 2
name= 'gradient_boosting'
with open('../confs/models.yaml', 'r') as file:
    configurations = yaml.safe_load(file)
factory = ModelFactory(configurations, X_train, y_train, train_scores)
start = time.time()
model = factory.get_model(name)
model.train()
score = model.evaluate(X_test)
# model.save(test_data)
end = time.time()
print(f'{model.name}={score} in {numpy.round((end-start), 2)}s')

In [ ]:
%load_ext autoreload
%autoreload 2
save_model = True
with open('../confs/models.yaml', 'r') as file:
    configurations = yaml.safe_load(file)
factory = ModelFactory(configurations, X_train, y_train, train_scores)
lines = []
for model in factory.get_models():
    start = time.time()
    model.train()
    score = model.evaluate(X_test)
    end = time.time()
    print(f'{model.name}={score} in {numpy.round((end-start), 2)}s')
    lines.append(eval_model(model, save_model))
df = pandas.DataFrame(lines)
df = df.sort_values(by=['score'], ascending=False)

In [ ]:
print(df)